In [2]:
# Load in the dependencies

%load_ext autoreload
%autoreload 2
from nn import nn, io, preprocess
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

I will first read in the data. We then have to make the negative examples the same sequence lengths as the positive examples. All of the positive examples have 17bp so I'll just loop through the negative sequences and create 17bp sub-sequences. 

In [3]:
positives = io.read_text_file("data/rap1-lieb-positives.txt")
negatives = io.read_fasta_file("data/yeast-upstream-1k-negative.fa")


# Make the negative sequences the same length as the positive sequences 
# We see that each of the positive sequences is 17bp 

seq_length = len(positives[0])
negative_trimmed = []
for s in negatives:
    seq = str()
    for i in range(len(s)):
        seq = seq + s[i] 
        
        if i > 0 and i % seq_length == 0:
            negative_trimmed.append(seq)
            seq = str()
    
# Combine and create labels
sequences = positives + negative_trimmed
labels = [True] * len(positives) + [False] * len(negative_trimmed)

# ensure each example has proper number of base pairs
sequences = [s[:seq_length] for s in sequences]

# Print counts
print("Positive examples:", len(positives))
print("Negative examples:", len(negative_trimmed))

Positive examples: 137
Negative examples: 183296


We then see that we have to subsample such that we equal number of examples for both positive and negative sequences. I chose to implement a function that upsamples the smaller class. I think this makes sense here because there are so few positive examples - so if I were to downsample the bigger class, there would be too few examples to train (fewer after splitting between training and validation). This will increase the number of positive examples and overall number of labeled examples available. 

In [4]:
seqs, labels = preprocess.sample_seqs(np.array(sequences), np.array(labels))

Now we will one hot encode the sequences. 

In [5]:
X = preprocess.one_hot_encode_seqs(seqs)

In [6]:
X

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

Now, we can split the data into training and testing sets. I just use the sklearn train_test_split function to do this. 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, np.array(labels), test_size = 0.2, random_state = 0)

Now, I create a tuning grid with different hyperparameter values. I will loop through all of the different combinations, fit a model and then select the best model. 

In [11]:
lr_grid = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
batch_grid = [10, 20, 50, 150, 200, 250, 300, 500, 750, 1000]
epochs_grid = [5, 10, 20, 30, 40, 50, 100, 200, 300, 500]

comb_array = np.array(np.meshgrid(lr_grid, batch_grid, epochs_grid)).T.reshape(-1, 3)
layers = [{"input_dim": 68, "output_dim": 34, "activation": "sigmoid"},
          {"input_dim": 34, "output_dim": 17, "activation": "sigmoid"},
          {"input_dim": 17, "output_dim": 1, "activation": "sigmoid"},]
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
tune_grid = []
count = 0
for i in comb_array:
    count += 1 
    lr = i[0]
    batch_size = i[1]
    epochs = int(i[2])
    net = nn.NeuralNetwork(layers, lr = lr, seed = 42, batch_size = batch_size, epochs = epochs, loss_function = "bce")
    train_losses, val_losses = net.fit(X_train, y_train, X_test, y_test)
    # We want to save the test loss for the last epoch
    test_loss = val_losses[-1]
    tune_grid.append([lr, batch_size, epochs, test_loss])


In [277]:
net = nn.NeuralNetwork(layers, lr = 0.001, seed = 42, batch_size = 10, epochs = 5, loss_function = "bce")
train_losses, val_losses = net.fit(X_train, y_train, X_test, y_test)
# Plot losses
fig, ax = plt.subplots(1, 2, figsize = (12, 4))
ax[0].plot(range(20), train_losses)
ax[0].set_title("Training")
ax[1].plot(range(20), val_losses)
ax[1].set_title("Validation")

ValueError: cannot reshape array of size 68 into shape (1,1)

In [ ]:
# Compute accuracy
pred = (net.predict(X_val) >= 0.5).astype(int)
print("Accuracy on validation set": {np.sum(pred == y_val) / len(y_val)}")